첫 샘플의 베스트

"reindeer mistletoe elf gingerbread family advent scrooge chimney fireplace ornament"

In [15]:
import gc ## for memory
import os
import copy
from math import exp
from collections import Counter
from typing import List, Optional, Union

import numpy as np
import pandas as pd
import transformers
import torch

from kaggle_evaluate import PerplexityCalculator

In [2]:
df_sample = pd.read_csv("sample_submission.csv")

In [3]:
df_sample.text[1].split()

['advent',
 'chimney',
 'elf',
 'family',
 'fireplace',
 'gingerbread',
 'mistletoe',
 'ornament',
 'reindeer',
 'scrooge',
 'walk',
 'give',
 'jump',
 'drive',
 'bake',
 'the',
 'sleep',
 'night',
 'laugh',
 'and']

## 그리디 서치

그리디 서치는 최적 방법을 산출하기 어렵지만, 계산이 간단하다. 한 번 해보자.

### Starter 생성 : GPT

In [4]:
starter = {
0 : "reindeer mistletoe elf gingerbread family advent scrooge chimney fireplace ornament",

1 : "reindeer mistletoe elf gingerbread family advent scrooge chimney fireplace ornament walk jump drive bake give sleep laugh the night and",

2 : "yuletide cheer gifts holiday carol decorations magi polar grinch nutcracker sleigh chimney workshop stocking holly ornament jingle beard naughty nice",

3 : "yuletide cheer holiday carol gifts decorations polar grinch nutcracker magi sleigh chimney workshop stocking holly ornament jingle beard naughty nice sing relax eat cheer unwrap visit of the and is",

4 : "poinsettia candle snowglobe peppermint hohoho eggnog chocolate fruitcake candy puzzle doll toy game workshop wrapping bow card greeting season joy merry peace hope believe dream wonder angel cookie milk wreath night star fireplace wish paper the to and of in have not as you that it with we from kaggle",

5 : "reindeer mistletoe elf gingerbread family advent scrooge chimney fireplace ornament walk jump drive bake give sleep laugh the night and yuletide cheer gifts holiday carol decorations magi polar grinch nutcracker sleigh chimney workshop stocking holly ornament jingle beard naughty nice sing relax eat cheer unwrap visit of the and is poinsettia candle snowglobe peppermint hohoho eggnog chocolate fruitcake candy puzzle doll toy game workshop wrapping bow card greeting season joy merry peace hope believe dream wonder angel cookie milk wreath night star fireplace wish paper the to and of in have not as you that it with we from kaggle"
}

In [5]:
df_sample.text.str.split().map(lambda x : len(x))

0     10
1     20
2     20
3     30
4     50
5    100
Name: text, dtype: int64

In [6]:
for i in range(5) :
    print(set(df_sample.text[i+1].split()) == set(list(starter.values())[i+1].split()))

True
True
True
True
True


In [6]:
df_submission = pd.DataFrame({"id":[0,1,2,3,4,5],"text":list(starter.values())})
df_submission.to_csv("gpt_starter.csv")

In [58]:
os.system("kaggle competitions submit -c santa-2024 -f gpt_starter.csv -m .")

100%|██████████| 1.50k/1.50k [00:00<00:00, 1.58kB/s]


Successfully submitted to Santa 2024 - The Perplexity Permutation Puzzle

0

`-` 그리디 알고리즘 적용

In [7]:
evaluatr = PerplexityCalculator("google/gemma-2-9b")

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [8]:
all_tokens = [l.split() for l in list(starter.values())]

1. 첫 번째 단어 빼고 후보군 구성
2. 각 단어들을 붙여가며 스코어 산출
3. 최선의 결과 산출, 단어 인덱스로 제거하여 후보군 갱신

In [10]:
best_word = []

for i, tokens in enumerate(all_tokens) :
    words = [tokens[0]]
    token_list = tokens[1:]
    
    for j in range(len(tokens)-1) :
        word_list = []
        for t in token_list :
            word_list.append(" ".join(words + [t]))
            
        perplexities = evaluatr.get_perplexity(word_list, batch_size = 1024)
        min_index = np.argmin(perplexities)
        words = words + [token_list[min_index]]
        
        del token_list[min_index]

    print(f"cycle rooped")
        
    best_word.append(words)

cycle rooped
cycle rooped
cycle rooped
cycle rooped
cycle rooped
cycle rooped


In [11]:
list(starter.values())[0]

'reindeer mistletoe elf gingerbread family advent scrooge chimney fireplace ornament'

In [12]:
df_submission = pd.DataFrame({"id":[0,1,2,3,4,5], "text":[" ".join(w) for w in best_word]})

In [15]:
df_submission.iloc[0, 1] = list(starter.values())[0]

In [17]:
df_submission.to_csv("greedy.csv")

In [18]:
os.system("kaggle competitions submit -c santa-2024 -f greedy.csv -m .")

100%|██████████| 1.50k/1.50k [00:00<00:00, 1.60kB/s]


Successfully submitted to Santa 2024 - The Perplexity Permutation Puzzle

0

`-` 그리디 서치 개선(방향 추가)

In [ ]:
best_word = []

for i, tokens in enumerate(all_tokens) :
    words = [tokens[0]]
    token_list = tokens[1:]
    
    for j in range(len(tokens)-1) :
        word_list = []
        additived_words = []
        
        for t in token_list :
            for k in range(len(words)+1) :
                words.insert(k, t)
                word_list.append(" ".join(words))
                
                del words[k]
                
                additived_words.append(t)
            
        perplexities = evaluatr.get_perplexity(word_list, batch_size = 1024)
        min_index = np.argmin(perplexities)
        words = word_list[min_index].split()
        
        token_list.remove(additived_words[min_index])

    print(f"cycle rooped")
        
    best_word.append(words)

In [ ]:
df_greedy2 = pd.read_csv("greedy2.csv", index_col=0)

In [ ]:
df_greedy2.iloc[0, 1] = "reindeer mistletoe elf gingerbread family advent scrooge chimney fireplace ornament"
df_greedy2.to_csv("greedy2_brute.csv")

In [ ]:
os.system("kaggle competitions submit -c santa-2024 -f greedy2_brute.csv -m .")

100%|██████████| 1.50k/1.50k [00:00<00:00, 1.60kB/s]


Successfully submitted to Santa 2024 - The Perplexity Permutation Puzzle

0